# Tools for big data - week 6 SQL vs NoSQL

Setting up sqlite3 db connections.

In [1]:
import sqlite3
import pprint
from pymongo import MongoClient
conn = sqlite3.connect('northwind.db')
c = conn.cursor()

In [2]:
# Select customer from customer table
c.execute("SELECT * FROM Customers WHERE CustomerID = 'ALFKI'")
c.fetchone()

('ALFKI',
 'Alfreds Futterkiste',
 'Maria Anders',
 'Sales Representative',
 'Obere Str. 57',
 'Berlin',
 None,
 '12209',
 'Germany',
 '030-0074321',
 '030-0076545')

## SQL exercise 1

In [3]:
# Gets all orders which the customer have made and the product info
c.execute("""SELECT CustomerALFKI.OrderID, CustomerALFKI.CustomerID, pd.ProductID FROM 
            ( SELECT * FROM Orders WHERE CustomerID = 'ALFKI' ) CustomerALFKI 
            INNER JOIN 'Order Details' od
            on CustomerALFKI.OrderID = od.OrderID
            INNER JOIN 'Products' pd
            on od.ProductID = pd.ProductID""")
c.fetchall()

[(10643, 'ALFKI', 28),
 (10643, 'ALFKI', 39),
 (10643, 'ALFKI', 46),
 (10692, 'ALFKI', 63),
 (10702, 'ALFKI', 3),
 (10702, 'ALFKI', 76),
 (10835, 'ALFKI', 59),
 (10835, 'ALFKI', 77),
 (10952, 'ALFKI', 6),
 (10952, 'ALFKI', 28),
 (11011, 'ALFKI', 58),
 (11011, 'ALFKI', 71)]

In [4]:
# Gets all orders which the customer have made
c.execute("""SELECT CustomerALFKI.OrderID, CustomerALFKI.CustomerID, od.ProductID FROM 
            ( SELECT * FROM Orders WHERE CustomerID = 'ALFKI' ) CustomerALFKI 
            INNER JOIN 'Order Details' od
            on CustomerALFKI.OrderID = od.OrderID""")
c.fetchall()

[(10643, 'ALFKI', 28),
 (10643, 'ALFKI', 39),
 (10643, 'ALFKI', 46),
 (10692, 'ALFKI', 63),
 (10702, 'ALFKI', 3),
 (10702, 'ALFKI', 76),
 (10835, 'ALFKI', 59),
 (10835, 'ALFKI', 77),
 (10952, 'ALFKI', 6),
 (10952, 'ALFKI', 28),
 (11011, 'ALFKI', 58),
 (11011, 'ALFKI', 71)]

## SQL exercise 2

In [5]:
# Gets all products which the customer have made and the product info
c.execute("""SELECT COUNT(*), CustomerALFKI.OrderID, CustomerALFKI.CustomerID FROM 
            ( SELECT * FROM Orders WHERE CustomerID = 'ALFKI' ) CustomerALFKI 
            INNER JOIN 'Order Details' od
            on CustomerALFKI.OrderID = od.OrderID
            INNER JOIN 'Products' pd
            on od.ProductID = pd.ProductID
            GROUP BY od.OrderID
            HAVING COUNT(*) > 1""")
c.fetchall()

[(3, 10643, 'ALFKI'),
 (2, 10702, 'ALFKI'),
 (2, 10835, 'ALFKI'),
 (2, 10952, 'ALFKI'),
 (2, 11011, 'ALFKI')]

Setting up mongodb client and db connections

In [6]:
# Setting up mongo client
client = MongoClient()
# Choosing db
db = client.Northwind
# Getting collections of interest
customers_collection = db['customers']
orders_collection = db['orders']
order_details_collection = db['order-details']
products_collection = db['products']
employees_collection = db['employees']

## Mongodb exercise 1
Aggregation for getting orders for customer ALFKI

In [7]:
orders_products_pipeline_for_customer = [
                                            { "$match": { "CustomerID": "ALFKI" } },
                                            { "$lookup" : 
                                             {
                                                 "from" : "order-details",
                                                 "localField" : "OrderID",
                                                 "foreignField" : "OrderID",
                                                 "as" : "orders_for_customer"
                                             }
                                            },
                                            { "$lookup" : 
                                             {
                                                 "from" : "products",
                                                 "localField" : "orders_for_customer.ProductID",
                                                 "foreignField" : "ProductID",
                                                 "as" : "products_for_order"
                                             }
                                            },
                                            { "$project" : 
                                             { 
                                                "CustomerID" : "$CustomerID", 
                                                "OrderID" : "$OrderID",
                                                "ProductsOrdered" : "$products_for_order.ProductID",
                                                "NumberOfOrders" : {"$size" : "$orders_for_customer" },
                                                "_id" : 0
                                             }
                                            }
                                        ]

In [8]:
pprint.pprint(list(orders_collection.aggregate(orders_products_pipeline_for_customer)))

[{'CustomerID': 'ALFKI',
  'NumberOfOrders': 3,
  'OrderID': 10643,
  'ProductsOrdered': [28, 39, 46]},
 {'CustomerID': 'ALFKI',
  'NumberOfOrders': 1,
  'OrderID': 10692,
  'ProductsOrdered': [63]},
 {'CustomerID': 'ALFKI',
  'NumberOfOrders': 2,
  'OrderID': 10702,
  'ProductsOrdered': [3, 76]},
 {'CustomerID': 'ALFKI',
  'NumberOfOrders': 2,
  'OrderID': 10835,
  'ProductsOrdered': [59, 77]},
 {'CustomerID': 'ALFKI',
  'NumberOfOrders': 2,
  'OrderID': 10952,
  'ProductsOrdered': [6, 28]},
 {'CustomerID': 'ALFKI',
  'NumberOfOrders': 2,
  'OrderID': 11011,
  'ProductsOrdered': [58, 71]}]


## Mongodb exercise 2
Aggregation for getting orders with minimum two uniq products.

In [9]:
uniq_orders_products_pipeline_for_customer = [
                                                    { "$match" : { "CustomerID": "ALFKI" } },
                                                    { "$lookup" : 
                                                     {
                                                         "from" : "order-details",
                                                         "localField" : "OrderID",
                                                         "foreignField" : "OrderID",
                                                         "as" : "orders_for_customer"
                                                     }
                                                    },
                                                    { "$lookup" : 
                                                     {
                                                         "from" : "products",
                                                         "localField" : "orders_for_customer.ProductID",
                                                         "foreignField" : "ProductID",
                                                         "as" : "products_for_order"
                                                     }
                                                    },
                                                    { "$project" : 
                                                     { 
                                                        "CustomerID" : "$CustomerID", 
                                                        "OrderID" : "$OrderID",
                                                        "ProductsOrdered" : "$products_for_order.ProductID",
                                                        "NumberOfOrders" : {"$size" : "$orders_for_customer" },
                                                        "_id" : 0
                                                     }
                                                    },
                                                    { "$match" : { "NumberOfOrders" : { "$gte" : 2 } } }
                                                ]

In [10]:
pprint.pprint(list(orders_collection.aggregate(uniq_orders_products_pipeline_for_customer_v2)))

[{'CustomerID': 'ALFKI',
  'NumberOfOrders': 3,
  'OrderID': 10643,
  'ProductsOrdered': [28, 39, 46]},
 {'CustomerID': 'ALFKI',
  'NumberOfOrders': 2,
  'OrderID': 10702,
  'ProductsOrdered': [3, 76]},
 {'CustomerID': 'ALFKI',
  'NumberOfOrders': 2,
  'OrderID': 10835,
  'ProductsOrdered': [59, 77]},
 {'CustomerID': 'ALFKI',
  'NumberOfOrders': 2,
  'OrderID': 10952,
  'ProductsOrdered': [6, 28]},
 {'CustomerID': 'ALFKI',
  'NumberOfOrders': 2,
  'OrderID': 11011,
  'ProductsOrdered': [58, 71]}]


In [11]:
# Getting customer ALFKI document
pprint.pprint(customers_collection.find_one({"CustomerID": "ALFKI"}))

{'Address': 'Obere Str. 57',
 'City': 'Berlin',
 'CompanyName': 'Alfreds Futterkiste',
 'ContactName': 'Maria Anders',
 'ContactTitle': 'Sales Representative',
 'Country': 'Germany',
 'CustomerID': 'ALFKI',
 'Fax': '030-0076545',
 'Phone': '030-0074321',
 'PostalCode': 12209,
 'Region': 'NULL',
 '_id': ObjectId('59dccb9700e57dd96b6148ed')}


In [28]:
#pprint.pprint(list(customers_collection.find()))

[{'Address': 'Avda. de la Constitución 2222',
  'City': 'México D.F.',
  'CompanyName': 'Ana Trujillo Emparedados y helados',
  'ContactName': 'Ana Trujillo',
  'ContactTitle': 'Owner',
  'Country': 'Mexico',
  'CustomerID': 'ANATR',
  'Fax': '(5) 555-3745',
  'Phone': '(5) 555-4729',
  'PostalCode': 5021,
  'Region': 'NULL',
  '_id': ObjectId('59dccb9700e57dd96b6148ec')},
 {'Address': 'Obere Str. 57',
  'City': 'Berlin',
  'CompanyName': 'Alfreds Futterkiste',
  'ContactName': 'Maria Anders',
  'ContactTitle': 'Sales Representative',
  'Country': 'Germany',
  'CustomerID': 'ALFKI',
  'Fax': '030-0076545',
  'Phone': '030-0074321',
  'PostalCode': 12209,
  'Region': 'NULL',
  '_id': ObjectId('59dccb9700e57dd96b6148ed')},
 {'Address': 'Mataderos  2312',
  'City': 'México D.F.',
  'CompanyName': 'Antonio Moreno Taquería',
  'ContactName': 'Antonio Moreno',
  'ContactTitle': 'Owner',
  'Country': 'Mexico',
  'CustomerID': 'ANTON',
  'Fax': 'NULL',
  'Phone': '(5) 555-3932',
  'PostalCode'

  'Phone': 'Spain',
  'PostalCode': 'NULL',
  'Region': 'Sevilla',
  '_id': ObjectId('59dccb9700e57dd96b614909'),
  'field11': 'NULL'},
 {'Address': 'Av. Brasil',
  'City': 442,
  'CompanyName': 'Gourmet Lanchonetes',
  'ContactName': 'André Fonseca',
  'ContactTitle': 'Sales Associate',
  'Country': '04876-786',
  'CustomerID': 'GOURL',
  'Fax': '(11) 555-9482',
  'Phone': 'Brazil',
  'PostalCode': 'SP',
  'Region': 'Campinas',
  '_id': ObjectId('59dccb9700e57dd96b61490a'),
  'field11': 'NULL'},
 {'Address': '2732 Baker Blvd.',
  'City': 'Eugene',
  'CompanyName': 'Great Lakes Food Market',
  'ContactName': 'Howard Snyder',
  'ContactTitle': 'Marketing Manager',
  'Country': 'USA',
  'CustomerID': 'GREAL',
  'Fax': 'NULL',
  'Phone': '(503) 555-7555',
  'PostalCode': 97403,
  'Region': 'OR',
  '_id': ObjectId('59dccb9700e57dd96b61490b')},
 {'Address': '5ª Ave. Los Palos Grandes',
  'City': 'Caracas',
  'CompanyName': 'GROSELLA-Restaurante',
  'ContactName': 'Manuel Pereira',
  'Contac

## Mongodb exercise 3
Aggregation for getting orders with minimum two uniq products.

In [49]:
from bson.code import Code

In [67]:
myMap = Code("function () {"
                "emit(this.Country, 1);"
            "}")

In [68]:
myReduce = Code("function (country, count) {"
                   "return Array.sum(count)"
               "}")

In [69]:
res = customers_collection.map_reduce(myMap, myReduce, "CountryForCustomer")

In [70]:
for od in res.find():
    print(od)

{'_id': 8022.0, 'value': 1.0}
{'_id': 13008.0, 'value': 1.0}
{'_id': 28001.0, 'value': 1.0}
{'_id': 28023.0, 'value': 1.0}
{'_id': 28034.0, 'value': 1.0}
{'_id': 41101.0, 'value': 1.0}
{'_id': 44000.0, 'value': 2.0}
{'_id': 59000.0, 'value': 1.0}
{'_id': 67000.0, 'value': 1.0}
{'_id': 69004.0, 'value': 1.0}
{'_id': 75012.0, 'value': 1.0}
{'_id': 75016.0, 'value': 1.0}
{'_id': 78000.0, 'value': 1.0}
{'_id': '02389-673', 'value': 1.0}
{'_id': '02389-890', 'value': 1.0}
{'_id': '04876-786', 'value': 1.0}
{'_id': '05432-043', 'value': 1.0}
{'_id': '05442-030', 'value': 1.0}
{'_id': '05454-876', 'value': 1.0}
{'_id': '05487-020', 'value': 1.0}
{'_id': '05634-030', 'value': 1.0}
{'_id': '08737-363', 'value': 1.0}
{'_id': 'Argentina', 'value': 3.0}
{'_id': 'Austria', 'value': 2.0}
{'_id': 'B-6000', 'value': 1.0}
{'_id': 'Belgium', 'value': 1.0}
{'_id': 'Canada', 'value': 3.0}
{'_id': 'Denmark', 'value': 2.0}
{'_id': 'Finland', 'value': 2.0}
{'_id': 'France', 'value': 2.0}
{'_id': 'Germany', '